In [1]:
import logging
import pandas as pd
from files.backend_codes import build_workflow, metadata_retrieval_node, sql_node, interpret_node, chart_node, processing_node, analize_step_node
import plotly.io as pio

# 基本的なロギングを設定
logging.basicConfig(level=logging.INFO)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Successfully loaded faiss with AVX2 support.
INFO:faiss:Failed to load GPU Faiss: name 'GpuIndexIVFFlat' is not defined. Will not load constructor refs for GPU indexes. This is only an error if you're trying to use GPU Faiss.


In [ ]:
state = {
    "user_input": "データベース内のテーブルの一覧を取得したい",
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
}
res = metadata_retrieval_node.func("データベース内のテーブルの一覧を取得したい", state)
print(res)


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


{'user_input': 'データベース内のテーブルの一覧を取得したい', 'messages': [], 'df_history': {}, 'sql_history': {}, 'metadata_answer': [{'データベース内のテーブルの一覧を取得したい': 'データベース内のテーブルの一覧を取得するためには、通常、SQLの`SHOW TABLES;`コマンドを使用します。このコマンドを実行すると、データベースに存在するすべてのテーブルの名前が表示されます。\n\n以下は、データベース内に存在するテーブルの概要です：\n\n1. **sales_data**\n   - **説明**: 売上データを格納するテーブルです。各レコードには、商品カテゴリ、商品名、販売金額、販売日が含まれています。\n   - **カラム**:\n     - `id`: レコードの一意な識別子（主キー）。\n     - `category`: 商品のカテゴリ名。\n     - `product_name`: 商品の名前。\n     - `sales`: 商品の販売金額。\n     - `date`: 商品が販売された日付（フォーマットはYYYY-MM-DD）。\n\n2. **products**\n   - **説明**: 商品マスタテーブルで、商品に関する基本情報が格納されています。\n   - **カラム**:\n     - `product_id`: 商品の一意な識別子（主キー）。\n     - `product_name`: 商品の名前。\n     - `category`: 商品のカテゴリ。\n     - `price`: 商品の価格。\n\n3. **users**\n   - **説明**: ユーザー情報を格納するテーブルです。ユーザーに関する基本的な情報が含まれています。\n   - **カラム**:\n     - `user_id`: ユーザーの一意な識別子（主キー）。\n     - `user_name`: ユーザーの名前。\n     - `age`: ユーザーの年齢。\n     - `prefecture`: ユーザーの居住都道府県。\n\n4. **reviews**\n   - **説明**: 商品レビューを格納するテ

In [5]:
state = {
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
    "analize_step": []
}
res = analize_step_node.func("ブランドの状況が知りたい", state)
print(res["analize_step"][0]["ブランドの状況が知りたい"])

INFO:root:analize_step_node: RAG情報を読み込み中 'ブランドの状況が知りたい'
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:root:analize_step_node: 生成AIが考えています・・・ 'ブランドの状況が知りたい'
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


ブランドの状況を把握するための分析計画を以下に示します。この計画では、提供されたテーブル情報を活用して、ブランドのパフォーマンスや顧客の反応を評価します。

### 分析計画

1. **目的の明確化**
   - ブランドの状況を把握するために、以下の観点から分析を行う。
     - 売上のトレンド
     - 商品の評価（レビュー）
     - ユーザーの属性（年齢、都道府県）

2. **データ収集**
   - 各テーブルから必要なデータを抽出する。
     - `sales_data` テーブルから、商品カテゴリ、商品名、販売金額、販売日を取得。
     - `reviews` テーブルから、商品ID、評価、コメントを取得。
     - `products` テーブルから、商品ID、商品名、カテゴリ、価格を取得。
     - `users` テーブルから、ユーザー名、年齢、都道府県を取得。

3. **データ分析**
   - **売上分析**
     - 商品カテゴリごとの売上金額を集計し、トレンドを可視化（例：月別売上推移）。
     - 売上が高い商品やカテゴリを特定し、ブランドの強みを把握。
   
   - **レビュー分析**
     - 商品ごとの平均評価を計算し、評価の高い商品を特定。
     - 評価の分布を分析し、顧客の満足度を把握。
     - コメント分析を行い、顧客の意見や要望を抽出（テキストマイニング手法を使用）。

   - **ユーザー分析**
     - ユーザーの年齢層や都道府県別の購入傾向を分析し、ターゲット市場を特定。
     - 年齢層ごとの評価傾向を分析し、特定の年齢層に人気のある商品を把握。

4. **結果の可視化**
   - 売上トレンドや評価分布をグラフやチャートで可視化し、理解しやすい形で報告。
   - ユーザー属性に基づく分析結果をダッシュボード形式でまとめ、関係者に共有。

5. **結論と提案**
   - 分析結果に基づいて、ブランドの強みや改善点を明確にし、今後の戦略に関する提案を行う。
   - 例えば、特定の年齢層に向けたマーケティング施策や、評価の低い商品の改善策など。

### まとめ
この分析計画を通じて、ブランドの状況を多角的に評価し、データに基づい

In [32]:
#SQL取得テスト
state = {
    "user_input": "データベース内のテーブルの一覧を取得したい",
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [{'カテゴリごとの売上の合計は？': [{'category': '食品', 'total_sales': 31500}, {'category': '飲料', 'total_sales': 55000}]}],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
}
res = sql_node.func("カテゴリごとの売上の合計は？", state)
df_history = res["df_history"]
dfs = {}
for entry in df_history:
    for question, data in entry.items():
        # DataFrame化
        df = pd.DataFrame(data)
        dfs[question] = df  # 要件ごとにDataFrameを辞書に格納
dfs[question]

INFO:root:sql_node: 要件を処理中: 'カテゴリごとの売上の合計は？'
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


,category,total_sales
0,食品,31500
1,飲料,55000


In [2]:
#解釈テスト
state = {
    "user_input": "データベース内のテーブルの一覧を取得したい",
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [{'カテゴリごとの売上の合計は？': [{'category': '食品', 'total_sales': 31500}, {'category': '飲料', 'total_sales': 55000}]}],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
}
res = interpret_node.func("カテゴリごとの売上の合計は？", state)
res["interpretation_history"]

INFO:root:interpret_node: df_historyの読み込み開始・・・ 'カテゴリごとの売上の合計は？'
INFO:root:interpret_node: 生成AIが考えています・・・ 'カテゴリごとの売上の合計は？'


{'カテゴリごとの売上の合計は？': [{'category': '食品', 'total_sales': 31500}, {'category': '飲料', 'total_sales': 55000}]}


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


[{'カテゴリごとの売上の合計は？': 'データから読み取れる特徴は、食品と飲料のカテゴリにおける売上の合計です。具体的には、食品の売上は31,500円、飲料の売上は55,000円となっています。これにより、飲料カテゴリの売上が食品カテゴリよりも大幅に高いことがわかります。全体として、飲料がより人気であるか、需要が高いことを示唆しています。'}]

In [2]:
#グラフ化テスト
#解釈テスト
state = {
    "user_input": "データベース内のテーブルの一覧を取得したい",
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [
        {'マスタの中味は？':
         [{"name":'田仲', "性別":"男"}, {"name":"鈴木", "性別":"女"}]
         },
        {'カテゴリごとの売上の合計は？': 
         [{'category': '食品', 'total_sales': 31500}, {'category': '飲料', 'total_sales': 55000}]
         },
    ],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
}  
res = chart_node.func("カテゴリごとの売上の合計は？", state)
for chart in state["chart_history"]:
    for task, plotly_json in chart.items():
        print(f"--- {task} ---")
        fig = pio.from_json(plotly_json)
        fig.show()


INFO:root:chart_node: df_historyの読み込み開始・・・ 'カテゴリごとの売上の合計は？'
c:\Users\BR18126\analytics_ai\files\backend_codes.py:340: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
INFO:root:chart_node: 生成AIが考えています・・・ 'カテゴリごとの売上の合計は？'




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


現在のタスクは、カテゴリごとの売上の合計を視覚化することです。利用可能なデータは `df1` で、カテゴリとその売上合計が含まれています。最適なグラフは、カテゴリごとの売上を示す棒グラフが適していると考えます。

Action: python_repl_ast  
Action Input: 
```python
import plotly.express as px
import pandas as pd

# df1のデータを作成
data = {
    'category': ['食品', '飲料'],
    'total_sales': [31500, 55000]
}
df1 = pd.DataFrame(data)

# 棒グラフを作成
fig = px.bar(df1, x='category', y='total_sales', title='カテゴリごとの売上の合計', labels={'total_sales': '売上合計', 'category': 'カテゴリ'})
fig.show()
```



Observation: 
Thought:

INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


Thought: 棒グラフを作成するためのコードを実行しましたが、グラフを表示するのではなく、JSON形式で出力する必要があります。したがって、`fig.to_json()`を呼び出してJSON文字列を取得し、出力します。

Action: python_repl_ast  
Action Input: 
```python
import plotly.express as px
import pandas as pd

# df1のデータを作成
data = {
    'category': ['食品', '飲料'],
    'total_sales': [31500, 55000]
}
df1 = pd.DataFrame(data)

# 棒グラフを作成
fig = px.bar(df1, x='category', y='total_sales', title='カテゴリごとの売上の合計', labels={'total_sales': '売上合計', 'category': 'カテゴリ'})

# JSON形式で出力
fig_json = fig.to_json()
print(fig_json)
```

Observation: {"data":[{"hovertemplate":"カテゴリ=%{x}\u003cbr\u003e売上合計=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"","orientation":"v","showlegend":false,"textposition":"auto","x":["食品","飲料"],"xaxis":"x","y":{"dtype":"i4","bdata":"DHsAANjWAAA="},"yaxis":"y","type":"bar"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscal

INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO:root:chart_node: Agent response: {'input': "\n    あなたはPythonプログラミングとデータ可視化の専門家です。\n    現在のタスクと文脈を踏まえて、最適なデータを選択し、最適なグラフを作成してください。\n\n    【現在のタスク】\n    カテゴリごとの売上の合計は？\n    \n    【ユーザーの全体的な質問の文脈】\n    \n\n    【利用可能なデータの内容】\n    \n                    # df0\n                    ## 内容:【マスタの中味は？】に関するデータ    \n                    ## dfの列情報: ['name', '性別']\n                    ## dfの最初の5行:\nname 性別\n  田仲  男\n  鈴木  女\n                    \n\n                    # df1\n                    ## 内容:【カテゴリごとの売上の合計は？】に関するデータ    \n                    ## dfの列情報: ['category', 'total_sales']\n                    ## dfの最初の5行:\ncategory  total_sales\n      食品        31500\n      飲料        55000\n                    \n\n    最適なインタラクティブグラフを `plotly.express` (例: `px`) を使用して生成してください。\n    生成したFigureオブジェクトを `fig` という変数に格納し、その後 `fig.to_js

I now know the final answer.

Final Answer: 
```json
{"data":[{"hovertemplate":"カテゴリ=%{x}\u003cbr\u003e売上合計=%{y}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#636efa","pattern":{"shape":""}},"name":"","orientation":"v","showlegend":false,"textposition":"auto","x":["食品","飲料"],"xaxis":"x","y":{"dtype":"i4","bdata":"DHsAANjWAAA="},"yaxis":"y","type":"bar"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.111

RuntimeError: 生成されたグラフのJSONが無効です: Expecting value: line 1 column 1 (char 0)

In [3]:
state = {
    "user_input": "データベース内のテーブルの一覧を取得したい",
    "messages": [],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [
        {'マスタの中味は？':
         [{"name":'田仲', "性別":"男"}, {"name":"鈴木", "性別":"女"}]
         },
        {'カテゴリごとの売上の合計は？': 
         [{'category': '食品', 'total_sales': 31500}, {'category': '飲料', 'total_sales': 55000}]
         },
    ],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
} 
res = processing_node.func("売上金額の合計は？", state)


c:\Users\BR18126\analytics_ai\files\backend_codes.py:441: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
INFO:root:dataprocess_node: 生成AIが考えています・・・ '売上金額の合計は？'




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


売上金額の合計を求めるためには、df1の'total_sales'列の合計を計算する必要があります。最終的な結果は'result'というDataFrameに格納します。

Action: python_repl_ast  
Action Input: result = df1['total_sales'].sum()  
Observation: 
Thought:

INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


最終的な結果をDataFrameとして'result'に格納する必要がありますので、合計値をDataFrameに変換します。

Action: python_repl_ast  
Action Input: result = pd.DataFrame({'total_sales_sum': [df1['total_sales'].sum()]})  
Observation: 
Thought:

INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO:root:processed_node: Agent response: {'input': "\n    あなたはPythonプログラミングの専門家です。\n    現在のタスクと文脈を踏まえて、最適なデータを選択し、求められたデータ加工を行ってください。\n    データ加工後の結果は必ず DataFrameとし、'result'という名前の変数に代入して返却してください。\n\n    【現在のタスク】\n    売上金額の合計は？\n    \n    【ユーザーの全体的な質問の文脈】\n    \n\n    【利用可能なデータの内容】\n    \n                    # df0\n                    ## 内容:【マスタの中味は？】に関するデータ    \n                    ## dfの列情報: ['name', '性別']\n                    ## dfの最初の5行:\nname 性別\n  田仲  男\n  鈴木  女\n                    \n\n                    # df1\n                    ## 内容:【カテゴリごとの売上の合計は？】に関するデータ    \n                    ## dfの列情報: ['category', 'total_sales']\n                    ## dfの最初の5行:\ncategory  total_sales\n      食品        31500\n      飲料        55000\n                    \n\n    実行すべきPythonコードのみを考えてください。\n    最終的な結果は'result'という名称のDat

I now know the final answer.  
Final Answer: DataFrame `result` contains the total sales sum.

> Finished chain.


In [ ]:
from langchain_core.messages import HumanMessage
logging.basicConfig(level=logging.INFO)
workflow = build_workflow()

# 例：ユーザー入力から初期状態を作って実行
state = {
    "messages": [HumanMessage(content="カテゴリごとの売上の合計を出してグラフにして解釈出して？")],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
    "analize_step": []
}

result, memory = workflow.invoke(state)

In [11]:
from langchain_core.messages import HumanMessage
logging.basicConfig(level=logging.INFO)
workflow = build_workflow()

# 例：ユーザー入力から初期状態を作って実行
state = {
    "messages": [HumanMessage(content="データベース内のテーブルの一覧を取得したい")],
    "task_description": [],
    "metadata_answer": [],
    "df_history": [],
    "sql_history": [],
    "interpretation_history": [],
    "chart_history": [],
    "analize_step": []
}

result, memory = workflow.invoke(state)

 supervisor: Thinking...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


 supervisor: Thinking...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


 supervisor: Thinking...


...

 supervisor: Thinking...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


 supervisor: Thinking...


INFO:httpx:HTTP Request: POST https://kh-aigrd-openai-prd-02.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

NameError: name 'memory' is not defined